In [1]:
features = [
    "Blue circle background",
    "Black cat body",
    "White chest patch",
    "Pink nose",
    "Yellow left eye",
    "Yellow right eye",
    "White left whisker",
    "White middle whisker",
    "White right whisker",
    "Pink left ear",
    "Pink right ear",
    "Black tail",
    "White left paw",
    "White right paw",
]

### Embedding based

In [2]:
from sentence_transformers import SentenceTransformer
import torch


embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

emb_features = embedding_model.encode(features)

def get_feature_to_edit_emb(instruction:str):
    
        
    embeded_ins = embedding_model.encode(instruction.split(" "))

    similarities: torch.Tensor = embedding_model.similarity(
        emb_features, embeded_ins 
    )

    min_sim = similarities.min()
    max_sim = similarities.max()
    similarities = (similarities - min_sim) / (max_sim - min_sim + 1e-8)

    
    similarities = [max(similarity) for similarity in similarities]
    
    print(f"\"{instruction}\"")
    for feature,similarity in sorted(list(zip(features,similarities)),key=lambda x : x[1],reverse=True):
        print(f"    {feature}:{similarity.item()}")



/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
instruction = "make the nose of the cat blue"
get_feature_to_edit_emb(instruction)
instruction = "make the cat wider"
get_feature_to_edit_emb(instruction)
instruction = "Make the mustache of the cat much longer"
get_feature_to_edit_emb(instruction)
instruction = "Change the position of the tail to the other side of the cat"
get_feature_to_edit_emb(instruction)
instruction = "Add brown pupils to the cat"
get_feature_to_edit_emb(instruction)

"make the nose of the cat blue"
    Pink nose:1.0
    Black cat body:0.8441013097763062
    Blue circle background:0.7163950800895691
    Black tail:0.5959071516990662
    White right paw:0.5101525783538818
    Yellow right eye:0.4712122678756714
    Pink right ear:0.4687386453151703
    White left paw:0.43886712193489075
    White right whisker:0.4374188482761383
    Pink left ear:0.418923020362854
    Yellow left eye:0.38957491517066956
    White chest patch:0.3763931393623352
    White middle whisker:0.3712800145149231
    White left whisker:0.29880210757255554
"make the cat wider"
    Black cat body:1.0
    White right paw:0.5830686688423157
    White left paw:0.4940696060657501
    Black tail:0.4595527946949005
    Pink nose:0.44862785935401917
    White middle whisker:0.280143141746521
    Blue circle background:0.27940523624420166
    White right whisker:0.26686593890190125
    Yellow right eye:0.24218058586120605
    White chest patch:0.23798537254333496
    Pink right ear:0.21

#### LLM-based

In [23]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)


def get_feature_to_edit_llm(instruction: str):

    model_insctruction = f"Among the following features :\n     {"\n    ".join([f for f in features])}\n Give me the 3 ones that are the most probable of being edited by the instruction : \n\"{instruction}\"\n ANSWER ONLY WITH THE 3 FEATURES, LISTED FROM MOST PROBABLE TO LEAST"
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": model_insctruction}]
    )
    most_probables = response.choices[0].message.content
    print(f"------------\"{instruction}\"--------------")
    print(most_probables)

In [24]:
instruction = "make the nose of the cat blue"
get_feature_to_edit_llm(instruction)
instruction = "make the cat wider"
get_feature_to_edit_llm(instruction)
instruction = "Make the mustache of the cat much longer"
get_feature_to_edit_llm(instruction)
instruction = "Change the position of the tail to the other side of the cat"
get_feature_to_edit_llm(instruction)
instruction = "Add brown pupils to the cat"
get_feature_to_edit_llm(instruction)

------------"make the nose of the cat blue"--------------
1. Pink nose
2. Blue circle background 
3. Black cat body
------------"make the cat wider"--------------
1. Black cat body
2. White left paw
3. White right paw
------------"Make the mustache of the cat much longer"--------------
1. White left whisker
2. White middle whisker
3. White right whisker
------------"Change the position of the tail to the other side of the cat"--------------
1. Black tail
2. Black cat body
3. White right paw
------------"Add brown pupils to the cat"--------------
1. Yellow right eye
2. Yellow left eye
3. Black cat body
